In [6]:
from functools import total_ordering
# total_ordering 需实现__lt__, __eq__两种方法

@total_ordering
class Rectangle(object):
    def __init__(self, w, h):
        self.w = w
        self.h = h
    
    def area(self):
        return self.w * self.h
    
    def __lt__(self, obj):
        print('in __lt__')
        return self.area() < obj.area()
    
#    def __le__(self, obj): #小于等于
#        print('in __le__')
#        return self.area() < obj.area()

    def __eq__(self, obj):
        print('in __eq__')
        return self.area() < obj.area()        

In [8]:
r1 = Rectangle(5, 3)
r2 = Rectangle(4, 4)

#r1 < r2 # 实际上调用 r1.__tl__(r2)

r1 <= r2 # 实际上调用的是__lt__, 由__eq__, __lt__推测出__le__的行为

in __lt__


True

In [10]:
class Circle(object):
    def __init__(self, r):
        self.r = r
    
    def area(self):
        return self.r ** 2 * 3.14

In [12]:
c1 = Circle(3)
print(r1 <= c1)
# print(c1 <= r1) 反之不行，Circle类无area接口

in __lt__
True


In [13]:
# 通过集成对接口进行约束；并简化方法的实现

from abc import ABCMeta, abstractmethod # 抽象基类

@total_ordering
class Shape(object):
    @abstractmethod #抽象方法，子类需将其实现, Shape为抽象类
    def area(self):
        pass

    def __lt__(self, obj):
        print('in __lt__')
        if not isinstance(obj, Shape): #注意继承与实例的关系
            raise TypeError('obj is not Shape')
        return self.area() < obj.area()
    
    def __eq__(self, obj):
        if not isinstance(obj, Shape): #注意继承与实例的关系
            raise TypeError('obj is not Shape')
        print('in __eq__')
        return self.area() < obj.area()
    
class Rectangle(Shape):
    def __init__(self, w, h):
        self.w = w
        self.h = h
    
    def area(self):
        return self.w * self.h
    
class Circle(object):
    def __init__(self, r):
        self.r = r
    
    def area(self):
        return self.r ** 2 * 3.14

In [15]:
print(c1 <= r1)
print(r1 > c1)

in __lt__
False
in __lt__
False


In [16]:
print(r1 > 1)

in __lt__


AttributeError: 'int' object has no attribute 'area'